### 순환 신경망

In [2]:
# 순환신경망(Recurrent Neural Network)은 지금까지 살펴본 네트워크와는 입력을 받아들이는 방식과 처리하는 방식에서 
# 차이가 있다. RNN은 순서가 있는 데이터를 입력으로 받고 같은 네트워크를 이용해서 변화하는 입력에 대한 출력을 얻어낸다.
# 순서가 있는 데이터로는 음악, 자연어, 날씨, 주가등 시간의 흐름에 따라 변화하고 그 변화가 의미를 갖는 데이터를 말한다.
# 이번 장에서는 그중 가장 범용적으로 쓰이는 자연어 처리에 RNN을 사용하는 방법을 알아보기로 한다.

In [ ]:
# RNN은 일반적인 딥러닝 네트워크처럼 입력 X를 받아서 출력 Y를 반환하지만 추가로 되먹임 구조를 가진다는 차이가 있다.
# 되먹임 구조란 어떤 레이어의 출력을 다시 입력으로 받는 구조를 말한다. RNN의 구조를 풀어보면 입력이 X1,X2,X3으로
# 변할 때 같은 네트워크를 사용해서 출력인 Y1,Y2,Y3를 반환하고 있는데 꼭 염두에 둬야 할 점은 출력값이 다음 입력을 
# 받을 때의 RNN 네트워크에도 동일하게 전달되고 있다는 것이다. 즉 RNN 네트워크는 처음에는 X1을 입력으로 받고 그다음
# 에는 X2와 이전 단계 출력값인 Y1, 그다음에는 X3와 이전 단계 출력값인 Y2를 입력으로 받는다. 이과정에서 RNN 네트워크
# 는 동일하게 사용된다.
# RNN은 입력과 출력의 길이에 제한이 없다는 특징이 있다. 다양한 형태의 RNN으로는 
# ONE TO MANY : Image Captioning,  MANY TO ONE : Sentiment Classification
# MANY TO MANY : Machine Translation, Video Classification(Frame level) 